In [90]:
from collections import Counter
import numpy as np
import pandas as pd
import xgboost as xgb
from scipy.sparse import csr_matrix
from sklearn.model_selection import KFold
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [80]:
cd /Users/williamzhou/Documents/github/RussianHousing/data/best

/Users/williamzhou/Documents/github/RussianHousing/data/best


In [81]:
x_train = pd.read_csv('./x_train.csv')
y_train = pd.read_csv('./y_train.csv')
x_test  = pd.read_csv('./x_test.csv')
x_train_fill = pd.read_csv('./x_train_fill.csv')
x_test_fill = pd.read_csv('./x_test_fill.csv')
y_train_fill = pd.read_csv('./y_train.csv')

In [82]:
full_sq = x_test['full_sq']

In [83]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)

x_train_fill = np.array(x_train_fill)
y_train_fill = np.array(y_train_fill)
x_test_fill = np.array(x_test_fill)

In [84]:
SEED = 0
NFOLDS = 5
NTHREADS = 4

In [85]:
def get_oof(clf, x_train, y_train, x_test):
    ntrain = x_train.shape[0]
    ntest = x_test.shape[0]
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    
    
    kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=SEED).split(x_train)

    for i, (train_index, test_index) in enumerate(kf):
        print(len(train_index))
        print(len(test_index))
        x_tr = x_train[train_index]
        print(x_tr.shape)
        y_tr = y_train[train_index]
        print(y_train.shape)
        x_te = x_train[test_index]
        print(x_te.shape)
        
        print('Start training fold {}'.format(i))
        clf.train(x_tr, y_tr)
        print('Finish training fold {}'.format(i))
        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)
        print('Finish predicting fold {}'.format(i))
    oof_test[:] = oof_test_skf.mean(axis=0)
    
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)


In [86]:
class XgbWrapper(object):
    def __init__(self, seed=0, params=None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 1000)

    def train(self, x_train, y_train):
        dtrain = xgb.DMatrix(x_train, label=y_train)
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds)

    def predict(self, x):
        return self.gbdt.predict(xgb.DMatrix(x))

In [87]:
class SklearnWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)

In [88]:

xgb1_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'early_stopping_rounds':20,
    'silent': 0}

xgb2_params = {
    'eta': 0.04,
    'max_depth': ,
    'subsample': 1,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'early_stopping_rounds':20,
    'silent': 0}

xgb3_params = {
    'eta': 0.06,
    'max_depth': 5,
    'subsample': 1,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'early_stopping_rounds':20,
    'silent': 0}

et_params = {
    'n_jobs': 16,
    'n_estimators': 100,
    'max_features': 0.5,
    'max_depth': 12,
    'min_samples_leaf': 2,
}

rf_params = {
    'n_jobs': 16,
    'n_estimators': 100,
    'max_features': 0.2,
    'max_depth': 8,
    'min_samples_leaf': 2,
}

In [94]:
%%time
dtrain = xgb.DMatrix(x_train, label=y_train)

# xg1 = XgbWrapper(seed=SEED, params=xgb1_params)
# xg1_train, xg1_test = get_oof(xg1, x_train, y_train, x_test)
# print("XG1-CV: {}".format(mean_squared_error(y_train, xg1_train)**0.5))

# xg2 = XgbWrapper(seed=SEED, params=xgb2_params)
# xg2_train, xg2_test = get_oof(xg2, x_train, y_train, x_test)
# print("XG2-CV: {}".format(mean_squared_error(y_train, xg2_train)**0.5))

# xg3 = XgbWrapper(seed=SEED, params=xgb3_params)
# xg3_train, xg3_test = get_oof(xg3, x_train, y_train, x_test)

# xg4 = XgbWrapper(seed=SEED, params=xgb4_params)
# xg4_train, xg4_test = get_oof(xg4, x_train, y_train, x_test)

et = SklearnWrapper(clf=ExtraTreesRegressor, seed=SEED, params=et_params)
et_train , et_test =  get_oof(et, x_train_fill, y_train_fill, x_test_fill)
print("et-CV: {}".format(mean_squared_error(y_train, et_train)**0.5))

rf = SklearnWrapper(clf=RandomForestRegressor, seed=SEED, params=rf_params)
rf_train , rf_test =  get_oof(rf, x_train_fill, y_train_fill, x_test_fill)
print("rf-CV: {}".format(mean_squared_error(y_train, rf_train)**0.5))

24376
6095
(24376, 40)
(30471, 1)
(6095, 40)
Start training fold 0


/Users/williamzhou/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Finish training fold 0
Finish predicting fold 0
24377
6094
(24377, 40)
(30471, 1)
(6094, 40)
Start training fold 1
Finish training fold 1
Finish predicting fold 1
24377
6094
(24377, 40)
(30471, 1)
(6094, 40)
Start training fold 2
Finish training fold 2
Finish predicting fold 2
24377
6094
(24377, 40)
(30471, 1)
(6094, 40)
Start training fold 3
Finish training fold 3
Finish predicting fold 3
24377
6094
(24377, 40)
(30471, 1)
(6094, 40)
Start training fold 4
Finish training fold 4
Finish predicting fold 4
et-CV: 40463.3528966
24376
6095
(24376, 40)
(30471, 1)
(6095, 40)
Start training fold 0
Finish training fold 0
Finish predicting fold 0
24377
6094
(24377, 40)
(30471, 1)
(6094, 40)
Start training fold 1
Finish training fold 1
Finish predicting fold 1
24377
6094
(24377, 40)
(30471, 1)
(6094, 40)
Start training fold 2
Finish training fold 2
Finish predicting fold 2
24377
6094
(24377, 40)
(30471, 1)
(6094, 40)
Start training fold 3
Finish training fold 3
Finish predicting fold 3
24377
6094


In [76]:
X_train_stack = np.concatenate([x_train, 
                                pd.DataFrame(xg1_train),
#                                 pd.DataFrame(xg2_train),
#                                 pd.DataFrame(xg3_train),
#                                 pd.DataFrame(xg4_train),
#                                 pd.DataFrame(et_train),
#                                 pd.DataFrame(rf_train),
                               ], axis=1)
xgtrain_stack = xgb.DMatrix(X_train_stack,y_train)


X_test_stack = np.concatenate([x_test, 
                                pd.DataFrame(xg1_test),
#                                 pd.DataFrame(xg2_test),
#                                 pd.DataFrame(xg3_test),
#                                 pd.DataFrame(xg4_test),
#                                 pd.DataFrame(et_test),
#                                 pd.DataFrame(rf_test),
                              ], axis=1)

xgtest_stack = xgb.DMatrix(X_test_stack)

In [77]:
final_xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0}

cv_output = xgb.cv(final_xgb_params, xgtrain_stack, 
                       num_boost_round=1000, 
                       early_stopping_rounds=20,
                       verbose_eval=50, show_stdv=False)
test_rmse = cv_output.loc[len(cv_output)-1,'test-rmse-mean']
print(test_rmse)  
num_boost_rounds = len(cv_output)
model = xgb.train(dict(final_xgb_params, silent=0), xgtrain_stack, num_boost_round=num_boost_rounds)


[0]	train-rmse:133932	test-rmse:133945
[50]	train-rmse:38665.3	test-rmse:40401.6
[100]	train-rmse:36197.1	test-rmse:39221.8
39204.3385417


In [78]:
y_predict = model.predict(xgtest_stack)
y_predic_all_sq = (y_predict)*full_sq
df_sub = pd.DataFrame({'id': range(30474,30474+7662), 'price_doc': y_predic_all_sq})
df_sub.to_csv('./sub.csv', index=False)
df_sub.head()

,id,price_doc
0,30474,5.328497e+06
1,30475,8.002381e+06
2,30476,5.092646e+06
3,30477,5.612750e+06
4,30478,4.817529e+06
